# nnUNetv2

In [1]:
import os
import pandas as pd
from pathlib import Path

DATASET_ID = '369'

SRC_FOLDER = os.path.join(
    Path().resolve().parent,
    'nnunet'
)

os.environ['nnUNet_raw'] = f"{SRC_FOLDER}/nnUNet_raw"
os.environ['nnUNet_preprocessed'] = f"{SRC_FOLDER}/nnUNet_preprocessed"
os.environ['nnUNet_results'] = f"{SRC_FOLDER}/nnUNet_results"

## 1 - Training

### 1.1 - nnUNet Plan and Preprocess

In [ ]:
!nnUNetv2_plan_and_preprocess -d 369 --verify_dataset_integrity -pl nnUNetPlannerResEncM

Fingerprint extraction...
Dataset369
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer

####################
verify_dataset_integrity Done. 
If you didn't see any error messages then your dataset is most likely OK!
####################

Experiment planning...
Attempting to find 3d_lowres config. 
Current spacing: [2.5        0.82681653 0.82681653]. 
Current patch size: (40, 224, 192). 
Current median shape: [ 97.         497.08737864 497.08737864]
Attempting to find 3d_lowres config. 
Current spacing: [2.5        0.85162102 0.85162102]. 
Current patch size: (40, 224, 192). 
Current median shape: [ 97.         482.60910548 482.60910548]
Attempting to find 3d_lowres config. 
Current spacing: [2.5        0.87716966 0.87716966]. 
Current patch size: (40, 224, 192). 
Current median shape: [ 97.         468.55252959 468.55252959]
Attempting to find 3d_lowres config. 
Current spacing: [2.5        0.90348475 0.90348475]. 
Current patch size: (40, 224, 192). 

### 1.2 - nnUNet 2D Model

In [ ]:
!nnUNetv2_train 369 '2d' 4 --npz -p nnUNetResEncUNetMPlans

Streaming output truncated to the last 5000 lines.
2024-10-23 12:13:39.590968: train_loss -0.9019
2024-10-23 12:13:39.598417: val_loss -0.8223
2024-10-23 12:13:39.608025: Pseudo dice [0.8929, 0.9386, 0.864, 0.8729, 0.7463, 0.7357, 0.9802, 0.9055, 0.9465, 0.9771, 0.9145, 0.8128]
2024-10-23 12:13:39.614017: Epoch time: 52.92 s
2024-10-23 12:13:39.618583: Yayy! New best EMA pseudo Dice: 0.866
2024-10-23 12:13:45.161760: 
2024-10-23 12:13:46.345123: Epoch 294
2024-10-23 12:13:46.348363: Current learning rate: 0.00731
2024-10-23 12:14:37.542452: train_loss -0.9005
2024-10-23 12:14:37.550178: val_loss -0.796
2024-10-23 12:14:37.559252: Pseudo dice [0.8678, 0.9147, 0.8609, 0.8465, 0.7041, 0.727, 0.9799, 0.9005, 0.9443, 0.9772, 0.9058, 0.818]
2024-10-23 12:14:37.581029: Epoch time: 52.39 s
2024-10-23 12:14:37.588135: Yayy! New best EMA pseudo Dice: 0.8665
2024-10-23 12:14:42.879934: 
2024-10-23 12:14:42.883454: Epoch 295
2024-10-23 12:14:42.885938: Current learning rate: 0.0073
2024-10-23 12:1

### 1.3 - nnUNet 3D Model

In [ ]:
!nnUNetv2_train 369 '3d_fullres' 0 --npz -p nnUNetResEncUNetMPlans

Using device: cuda:0
/usr/local/lib/python3.10/dist-packages/nnunetv2/training/nnUNetTrainer/nnUNetTrainer.py:164: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.grad_scaler = GradScaler() if self.device.type == 'cuda' else None

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

2024-10-29 05:36:38.391827: do_dummy_2d_data_aug: True
2024-10-29 05:36:39.051674: Using splits from existing split file: /content/gdrive/MyDrive/cs701-deep-learning-and-computer-vision/nnUNet_preprocessed/Dataset369/splits_final.json
2024-10-29 05:36:39.592702: The split file con

## 2 - Model Prediction/Inference

### 2.1 - nnUNet 2D Model

In [3]:
!nnUNetv2_find_best_configuration 369 -c '2d' -f 4 -p nnUNetResEncUNetMPlans


***All results:***
nnUNetTrainer__nnUNetResEncUNetMPlans__2d: 0.8656203751558422

*Best*: nnUNetTrainer__nnUNetResEncUNetMPlans__2d: 0.8656203751558422

***Determining postprocessing for best model/ensemble***
Removing all but the largest foreground region did not improve results!
Results were improved by removing all but the largest component for 1. Dice before: 0.86625 after: 0.86864
Removing all but the largest component for 2 did not improve results! Dice before: 0.91824 after: 0.9052
Results were improved by removing all but the largest component for 3. Dice before: 0.87928 after: 0.88012
Removing all but the largest component for 4 did not improve results! Dice before: 0.84923 after: 0.76784
Results were improved by removing all but the largest component for 5. Dice before: 0.68505 after: 0.69625
Results were improved by removing all but the largest component for 6. Dice before: 0.68439 after: 0.68981
Results were improved by removing all but the largest component for 7. Dice be

In [ ]:
!nnUNetv2_predict -d Dataset369 -i ./nnUNet_raw/Dataset369/imagesTs -o ./nnUNet_inference/valPred369 -f  4 -tr nnUNetTrainer -c 2d -p nnUNetResEncUNetMPlans

In [ ]:
!nnUNetv2_apply_postprocessing -i ./nnUNet_inference/valPred369 -o ./nnUNet_inference/valPred369_PP -pp_pkl_file /Users/keithchiang/Documents/GitHub/CS701-Group-09-Project/src/nnunet/nnUNet_results/Dataset369/nnUNetTrainer__nnUNetResEncUNetMPlans__2d/crossval_results_folds_4/postprocessing.pkl -np 8 -plans_json /Users/keithchiang/Documents/GitHub/CS701-Group-09-Project/src/nnunet/nnUNet_results/Dataset369/nnUNetTrainer__nnUNetResEncUNetMPlans__2d/crossval_results_folds_4/plans.json

# 3 - Model Evaluation

In [6]:
import SimpleITK as sitk
import numpy as np
import os

def load_nifti_image(filepath: str) -> np.ndarray:
    image = sitk.ReadImage(filepath)
    return sitk.GetArrayFromImage(image)

def dice_coefficient(mask1: np.ndarray, mask2: np.ndarray) -> float:
    intersect = np.sum(mask1 * mask2)
    fsum = np.sum(mask1)
    ssum = np.sum(mask2)
    if fsum + ssum == 0:
        # Both masks are empty
        dice = 1.0
    else:
        dice = (2 * intersect) / (fsum + ssum)
    
    return dice

def calculate_dice_coefficient(pred_path: str, gt_path: str):
    """
    Calculate the Dice coefficients for each label in 3D images given the file paths of the prediction
    and ground truth images in .nii.gz format.
    Args:
        pred_path (str): Path to the predicted segmentation (.nii.gz).
        gt_path (str): Path to the ground truth segmentation (.nii.gz).
    Returns:
        dict: Dictionary mapping labels to their Dice coefficient.
    """
    # Load the predicted and ground truth images
    pred_image = load_nifti_image(pred_path)
    gt_image = load_nifti_image(gt_path)

    # Ensure that the shapes match
    if pred_image.shape != gt_image.shape:
        raise ValueError("Predicted and ground truth images must have the same shape.")

    # Get the union of labels in both images, excluding background label (0)
    labels = np.union1d(np.unique(pred_image), np.unique(gt_image))
    labels = labels[labels != 0]  # Exclude background label

    dice_scores = {}
    
    for label in labels:
        pred_mask = (pred_image == label).astype(np.uint8)
        gt_mask = (gt_image == label).astype(np.uint8)
        dice = dice_coefficient(pred_mask, gt_mask)
        dice_scores[label] = dice

    return dice_scores

In [28]:
pred_file_src_path = "./nnUNet_inference/valPred369_PP"
groundtruth_file_src_path = "./nnUNet_raw/Dataset369/labelsTs"

pred_src_files_list = sorted(os.listdir(pred_file_src_path), key=lambda x: int(x.split('.')[0]))
groundtruth_src_files_list = sorted(os.listdir(groundtruth_file_src_path), key=lambda x: int(x.split('_0000')[0]))

results_df = pd.DataFrame()

for i in range(len(pred_src_files_list)):
    CT_SCAN_ID = pred_src_files_list[i].split('.')[0]

    pred_file = os.path.join(pred_file_src_path, pred_src_files_list[i])
    groundtruth_file = os.path.join(groundtruth_file_src_path, groundtruth_src_files_list[i])

    dice_scores = calculate_dice_coefficient(pred_file, groundtruth_file)
    average_dice = round(np.mean(list(dice_scores.values())), 4)

    print(f"{pred_src_files_list[i]} - Average Dice Coefficient: {average_dice:.4f}")


    # formatting results into dataframe
    ct_scan_pred_df = pd.DataFrame(list(dice_scores.items()), columns=['label', 'dice_score'])
    ct_scan_pred_df['scan_id'] = CT_SCAN_ID
    
    # aggregating results across all scans
    if results_df.empty:
        results_df = ct_scan_pred_df
    else:
        results_df = pd.concat([results_df, ct_scan_pred_df], ignore_index=True)
    

041.nii.gz - Average Dice Coefficient: 0.9087
042.nii.gz - Average Dice Coefficient: 0.9077
043.nii.gz - Average Dice Coefficient: 0.8139
044.nii.gz - Average Dice Coefficient: 0.9272
045.nii.gz - Average Dice Coefficient: 0.7715
046.nii.gz - Average Dice Coefficient: 0.6303
047.nii.gz - Average Dice Coefficient: 0.9326
048.nii.gz - Average Dice Coefficient: 0.7687
049.nii.gz - Average Dice Coefficient: 0.8907
050.nii.gz - Average Dice Coefficient: 0.8748


In [44]:
# performance
### by Overall
overall_dice = results_df['dice_score'].mean()

### by Label
results_label_performance_df = results_df.groupby(['label'], as_index=False)['dice_score'].mean()

### by CT scan ID
results_ct_scan_id_performance_df = results_df.groupby(['scan_id'], as_index=False)['dice_score'].mean()

print(f"Overall Dice Coefficient:", round(overall_dice, 4))
print()
print("Dice Coefficient by Label:")
print(results_label_performance_df)
print()
print("Dice Coefficient by CT Scan ID:")
print(results_ct_scan_id_performance_df)

Overall Dice Coefficient: 0.8426

Dice Coefficient by Label:
    label  dice_score
0       1    0.670777
1       2    0.902765
2       3    0.835212
3       4    0.910975
4       5    0.743243
5       6    0.727363
6       7    0.972328
7       8    0.941091
8       9    0.950732
9      10    0.900352
10     11    0.870706
11     12    0.685786

Dice Coefficient by CT Scan ID:
  scan_id  dice_score
0     041    0.908716
1     042    0.907665
2     043    0.813912
3     044    0.927233
4     045    0.771540
5     046    0.630270
6     047    0.932634
7     048    0.768661
8     049    0.890712
9     050    0.874763
